In [315]:
from sqlite3 import connect
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
# get the absolute directory of the project
basic_dir = "../"

# get the environment file and load it
env_path = os.path.join(basic_dir, '.env')
load_dotenv(dotenv_path=env_path)

banking_data_db = f"{basic_dir}/database/banking_data.db"

In [3]:
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    base_url= OPENAI_BASE_URL,
    api_key = OPENAI_API_KEY,
    temperature=0,
)

## Primary assistant tools

In [154]:
def contact_rm_tool(user_id: str, appointment_date_time: Optional[str] = "") -> str:
    """
    Check the schedules of appoint with the relationship manager (RM) or schedule a new appointment with the RM.
    Args:
        date: Meeting date (YYYY-MM-DD).
        time: Meeting time (HH:MM, 24-hour format).

    Returns:
        information about user's appointment with their RM or the status of new appointment booking.

    """
    conn = connect(banking_data_db)
    query = "SELECT * FROM pending_appointments WHERE user_id = ?"
    df_appointments = pd.read_sql_query(query, conn, params=(user_id,)).sort_values("appointment_date_time").reset_index(drop=True)
    dict_appointments = df_appointments.to_dict(orient="records")

    if appointment_date_time == "": # user only needs to check the existing appointments
        if len(dict_appointments) == 0:
            return "The user has no appointment with the relationship manager."
        else:
            appointment_summary = "Below are the user's current appointments with the relationship manger: "
            for item in dict_appointments:
                appt_dt = datetime.strptime(item['appointment_date_time'], "%Y-%m-%d %H:%M:%S")
                appointment_summary += (
                    f"On {item['date'][:10]}, the user made an appointment with the relationship manager on "
                    f"{appt_dt.strftime('%A, %B %d, %Y at %I:%M %p')}."
                )
            return appointment_summary
    
    else:
        try:
            appointment_dt = datetime.strptime(appointment_date_time, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            return "The appointment date time format is invalid. Please use 'YYYY-MM-DD hh:mm:ss'."
        
        # Get time difference to today
        today = datetime.today()
        time_diff = appointment_dt - today
        
        # Check if appointment is in the past
        if time_diff.total_seconds() < 0:
            return "You cannot make an appointment in the past. Please select a future date that is at least one day later."
        
        # Check if appointment is less than one day away
        if time_diff < timedelta(days=1):
            return "The appointment time must be scheduled at least one day in advance. Please choose a later time."

        # Check conflicts with existing appointment
        for item in dict_appointments:
            existing_app_datetime = datetime.strptime(item['appointment_date_time'], "%Y-%m-%d %H:%M:%S")
            time_diff_existing = appointment_dt - existing_app_datetime
            if abs(time_diff_existing) < timedelta(hours=2):
                return (
                    f"You already have an appointment with your relationship manager on "
                    f"{existing_app_datetime.strftime('%A, %B %d, %Y at %I:%M %p')}. "
                    f"Your new appointment must be scheduled at least 2 hours earlier or later than this time."
                )
                
        # The appointment is OK to schedule
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO pending_appointments (date, user_id, appointment_date_time)
            VALUES (?, ?, ?)
            """,
            (today.strftime("%Y-%m-%d %H:%M:%S"), user_id, appointment_dt)
        )
        conn.commit()
        conn.close()

        return f"Your new appointment with your relationship manager is scheduled at {appointment_dt.strftime('%A, %B %d, %Y at %I:%M %p')}."

In [157]:
# contact_rm_tool(user_id = "AB123")
contact_rm_tool(user_id = "AB123", appointment_date_time = "2025-09-27 8:30:00")

'You already have an appointment with your relationship manager on Saturday, September 27, 2025 at 08:01 AM. Your new appointment must be scheduled at least 2 hours earlier or later than this time.'

## Account assistant tools

In [23]:
def check_account_history(user_id: str, start_date: str, end_date: str):
    """
    Check transaction history of user's saving account.
    :param user_id: the user's id
    :param start_date: the start date of the transaction history
    :param end_date: the end date of the transaction history
    :return: a summary transaction history from this user id
    """
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT saving_account FROM user WHERE user_id = ?", (user_id,))
    row = cursor.fetchone()
    if row is None:
        return "No user found with this ID."
    saving_account = row[0]
    if saving_account is None:
        return "The client has no saving account with the bank."

    # Extract the data in the queried period.
    query = f"SELECT * FROM {saving_account} WHERE date BETWEEN ? AND ? ORDER BY date"
    df_all = pd.read_sql_query(query, conn, params=(start_date, end_date)).sort_values("date").reset_index(drop=True)  # Sort trades by date (safety)
    conn.close()

    if df_all.empty:
        return "No transactions found within the specified date range."

    df_all["date"] = pd.to_datetime(df_all["date"])

    # Time span in months
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    months_span = max((end.year - start.year) * 12 + end.month - start.month, 1)

    # Income analysis
    df_income = df_all[df_all["transaction_category"] == "Income"]
    total_income = df_income["transaction_amount"].sum()
    highest_income = df_income.loc[df_income["transaction_amount"].idxmax()]
    lowest_income = df_income.loc[df_income["transaction_amount"].idxmin()]
    avg_income = total_income / months_span

    # Expense analysis
    df_expense = df_all[df_all["transaction_category"] == "Expense"]
    total_expense = -df_expense["transaction_amount"].sum()
    highest_expense = df_expense.loc[df_expense["transaction_amount"].idxmin()]
    lowest_expense = df_expense.loc[df_expense["transaction_amount"].idxmax()]
    avg_expense = total_expense / months_span

    # Build summary
    summary = f"**Transaction Summary from {start_date} to {end_date}**\n\n"

    summary += f"**Income Overview**\n"
    summary += f"- Total Income: ${total_income:,.2f}\n"
    summary += f"- Highest Income: ${highest_income['transaction_amount']:,.2f} on {highest_income['date'].date()} ({highest_income['description']})\n"
    if lowest_income is not None:
        summary += f"- Lowest Income (excluding interest): ${lowest_income['transaction_amount']:,.2f} on {lowest_income['date'].date()} ({lowest_income['description']})\n"
    else:
        summary += f"- No non-interest income found to determine lowest income.\n"
    if months_span > 1:
        summary += f"- Average Monthly Income: ${avg_income:,.2f}\n"

    summary += f"\n**Expense Overview**\n"
    summary += f"- Total Expense: ${total_expense:,.2f}\n"
    summary += f"- highest Expense: ${-highest_expense['transaction_amount']:,.2f} on {highest_expense['date'].date()} ({highest_expense['description']})\n"
    summary += f"- lowest Expense: ${-lowest_expense['transaction_amount']:,.2f} on {lowest_expense['date'].date()} ({lowest_expense['description']})\n"
    if months_span > 1:
        summary += f"- Average Monthly Expense: ${avg_expense:,.2f}\n"

    return {
        "summary": summary,
        "income_transactions": df_income.to_dict(orient="records"),
        "expense_transactions": df_expense.to_dict(orient="records")
    }

In [27]:
print(check_account_history("AB123", "2025-07-08", "2025-09-08")["summary"])

**Transaction Summary from 2025-07-08 to 2025-09-08**

**Income Overview**
- Total Income: $6,930.00
- Highest Income: $3,200.00 on 2025-07-23 (Salary Deposit)
- Lowest Income (excluding interest): $80.00 on 2025-08-09 (Refund from Vendor)
- Average Monthly Income: $3,465.00

**Expense Overview**
- Total Expense: $645.90
- highest Expense: $250.00 on 2025-08-07 (Car Maintenance)
- lowest Expense: $45.00 on 2025-08-13 (Mobile Bill)
- Average Monthly Expense: $322.95



In [218]:
def check_pending_transfer(user_id: str):
    """
    Check pending transfers of the user.
    Args:
        user_id: user's id

    Returns:
        Information about user's pending transfers.
    """
    # Have the user's saving account first.
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT saving_account FROM user WHERE user_id = ?", (user_id,))
    row = cursor.fetchone()
    if row is None:
        conn.close()
        cursor.close()
        return "No user found with this ID."
    saving_account = row[0]
    if saving_account is None:
        conn.close()
        cursor.close()
        return "The user has no saving account with the bank."

    # Have the pending tansfers
    query = "SELECT * FROM pending_transfers WHERE sender_account = ?"
    df_transfers = pd.read_sql_query(query, conn, params=(saving_account,)).sort_values(
        "transfer_date").reset_index(drop=True)
    dict_transfers = df_transfers.to_dict(orient="records")
    cursor.close()
    conn.close()
    
    if len(dict_transfers) == 0:
        return "The user has no pending transfers."
    else:
        transfer_summary = "Below are the user's pending transfers:\n"
        total_pending_transfer = 0
        for item in dict_transfers:
            transfer_dt = datetime.strptime(item['transfer_date'][:10], "%Y-%m-%d")
            transfer_summary += (
                f"- ${item['transfer_amount']:.2f} to {item['recipient_account']} "
                f"at {item['recipient_bank']} on {transfer_dt.strftime('%A, %B %d, %Y')}.\n"
            )
            total_pending_transfer += item['transfer_amount']
        transfer_summary += f"Total pending transfer amount is ${total_pending_transfer:.2f}."
        
        return transfer_summary


def transfer_fund(user_id: str, amount: float, recipient_account: str, recipient_bank: str, transfer_date: str):
    """
    Make a transfer from user's saving account to another account.
    To submit the transfer request successfully at the same day, the user must have sufficient money in the account.
    And the transfer amount should be no more than $3000 per transaction.
    Args:
        user_id: user's id
        amount: the amount of money user would like to transfer, in dollar
        recipient_account: the amount number of the recipient
        recipient_bank: the bank of the recipient's account
        transfer_date: the date in which the user would like to perform this transfer

    Returns:
        The status of the new transfer.
    """
    # Have the user's saving account first.
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT saving_account FROM user WHERE user_id = ?", (user_id,))
    row = cursor.fetchone()
    if row is None:
        conn.close()
        cursor.close()
        return "No user found with this ID."
    saving_account = row[0]
    if saving_account is None:
        conn.close()
        cursor.close()
        return "The user has no saving account with the bank."

    
    
    # Check if the transfer date is valid.
    try:
        transfer_dt = datetime.strptime(transfer_date, "%Y-%m-%d").date()
    except ValueError:
        cursor.close()
        conn.close()
        return "The transfer date format is invalid. Please use 'YYYY-MM-DD'."
    today = datetime.today().date()
    if transfer_dt < today:
        cursor.close()
        conn.close()
        return "Transfers can only be scheduled for today or a future date. Kindly update the transfer date to proceed."

    # Check if the transfer amount is valid
    if amount > 3000:
        cursor.close()
        conn.close()
        return "The maximum allowable amount per transaction is $3,000. To request a limit adjustment, please contact your relationship manager."
    if amount < 0:
        cursor.close()
        conn.close()
        return "Please input an appropriate transfer amount."

    # Check if the remaining balance (excluding today's pending transfers) is sufficient (only for today's transfer)
    query = f"SELECT balance FROM {saving_account} where date = (SELECT MAX(date) FROM {saving_account})"
    cursor.execute(query)
    current_balance = cursor.fetchone()[0]
    
    # Have the pending tansfers
    query = "SELECT * FROM pending_transfers WHERE sender_account = ?"
    df_transfers = pd.read_sql_query(query, conn, params=(saving_account,)).sort_values("transfer_date").reset_index(drop=True)
    dict_transfers = df_transfers.to_dict(orient="records")
    
    # Have today's sum of pending transfers
    today_pending_transfer = 0
    if len(dict_transfers) > 0:
        today = datetime.today().date()
        today_pending_transfer = sum(item["transfer_amount"]
                                     for item in dict_transfers
                                     if datetime.strptime(item['transfer_date'][:10], "%Y-%m-%d").date() == today)

    available_funds = current_balance - today_pending_transfer
    if transfer_dt == today and amount > available_funds:
        cursor.close()
        conn.close()
        return (
            f"Insufficient funds: your account does not currently hold enough balance to complete this transfer.\n"
            f"Available transferable amount today: ${available_funds:,.2f}.\n"
            "Please ensure adequate funds are available before proceeding."
        )
    # The transfer is OK to proceed
    else:
        cursor.execute(
            """
            INSERT INTO pending_transfers (date, sender_account, transfer_amount, recipient_account, recipient_bank, transfer_date)
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            (today.strftime("%Y-%m-%d"), saving_account, amount, recipient_account, recipient_bank, transfer_date)
        )
        conn.commit()
        cursor.close()
        conn.close()

        formatted_date = datetime.strptime(transfer_date, "%Y-%m-%d").strftime("%A, %B %d, %Y")
        confirmation_message = f"Your transfer of ${amount:,.2f} to account {recipient_account} at {recipient_bank} has been successfully scheduled for {formatted_date}."
        if transfer_dt > today:
            confirmation_message += " Kindly ensure that sufficient funds are available in your account on the scheduled transfer date."

        return confirmation_message

In [220]:
print(transfer_fund("AB123", 100, 'KCF', 'The bank', '2025-09-19'))
# print(transfer_fund("AB123"))

Your transfer of $100.00 to account KCF at The bank has been successfully scheduled for Friday, September 19, 2025.


In [221]:
print(check_pending_transfer("AB123"))

Below are the user's pending transfers:
- $1000.00 to KCF at The bank on Friday, September 19, 2025.
- $200.00 to KCF at The bank on Friday, September 19, 2025.
- $100.00 to KCF at The bank on Friday, September 19, 2025.
- $500.00 to UH928 at Pension Board on Friday, September 19, 2025.
- $1500.00 to D983234 at DP Bank on Friday, September 19, 2025.
- $2000.00 to KCF at The bank on Saturday, September 20, 2025.
- $2000.00 to KCF at The bank on Sunday, September 21, 2025.
Total pending transfer amount is $7300.00.


## Trading assistant tools

In [343]:
class stock_price(BaseModel):
    """
    Return the stock name and its current price.
    """
    stock_name : Optional[str] = Field(description = "the company name of the stock, not the ticker, not with '.com' or any appendix. English name only.")
    stock_price: Optional[float] = Field(description="the current price of the stock")

def get_current_price(stock: str):
    """
    Get the current stock price.
    Args:
        stock: the stock/security/share name

    Returns:
        the current stock price
    """
    search = TavilySearch(max_results=1, api_key=TAVILY_API_KEY)
    current_price = None
    query_listed = f"The stock market where {stock} is traded"
    response_listed = search.run(query_listed)
    print(response_listed["results"][0]["content"])

    result_listed = llm.invoke(
        f"This decribes the listed market of a stock: {response_listed['results'][0]['content']}."
        "If you think it is not listed in the US stock market, return the string of 'No'."
        "Otherwise, return the string of 'Yes'."
        "Only make judgement based on provided information, don't assume anything."
        "Only return the result in srting of 'Yes' or 'No'."
    )

    print(result_listed.content)

    if result_listed.content == 'No':
        no_listed_result = stock_price(stock_name = None, stock_price = None)
        return no_listed_result
    else: 
        query_price = f"Check the current share price of {stock} in the US stock market."
        response_price = search.run(query_price)
        if response_price["results"][0]["content"]:
            prompt_template = PromptTemplate.from_template(
                'Here is the latest information of a stock price: {info}, '
                'please extract the company English name as the stock name in string and the the stock price in float.'
                'For the stock name, use the English company name, not the ticker, not with "Inc", ".com", "Corporation" or any appendix. Return English name only.'
                'For example, use "Adobe", never use "Adobe Inc.", "Adobe.com" or "Adobe Corporation"'
                'For the stock price, if there are multiple numbers mentioned, use the one with highest probability as the stoke price. Return one float only.'
            )
            runnable = llm.with_structured_output(stock_price)
            chain = prompt_template | runnable
            final_response = chain.invoke({'info':response_price["results"][0]["content"]})
        print(final_response)
    return final_response

In [301]:
result = get_current_price("Locksheet Martin")

Its market capitalization was valued at US$109.83 billion at the end of 2019. ... Lockheed Martin is listed as the largest U.S. government contractor and
Yes
stock_name='Lockheed Martin Corporation' stock_price=475.16


In [302]:
print(result)

stock_name='Lockheed Martin Corporation' stock_price=475.16


In [305]:
result.stock_name

'Lockheed Martin Corporation'

In [223]:
@tool
def check_pending_order(user_id: str):
    """
    Check pending orders of the user.
    Args:
        user_id: user's id

    Returns:
        Information about user's pending orders.
    """
    # Have the user's trading account first.
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT trading_account FROM user WHERE user_id = ?", (user_id,))
    row = cursor.fetchone()
    if row is None:
        conn.close()
        cursor.close()
        return "No user found with this ID."
    trading_account = row[0]
    if trading_account is None:
        conn.close()
        cursor.close()
        return "The user has no trading account with the bank."

    # Have the pending orders
    query = "SELECT * FROM pending_orders WHERE trading_account = ?"
    df_orders = pd.read_sql_query(query, conn, params=(trading_account,)).reset_index(drop=True)
    dict_orders = df_orders.to_dict(orient="records")
    cursor.close()
    conn.close()

    if len(dict_orders) == 0:
        return "The user has no pending orders."
    else:
        buy_order_amount = 0
        order_summary = "Below are the user's pending orders (Daily Limited Orders):\n"
        for item in dict_orders:
            order_summary += (
                f"- {item['action']} order of {item['volume']} shares of {item['stock']} at ${item['unit_price']:.2f} per share "
                f"with trading fee of ${item['trading_fee']:.2f}, total transfer amount is ${item['total_amount']:.2f}.\n"
            )
            if item['type'] == "buy":
                buy_order_amount += item['total_amount']
            if buy_order_amount > 0:
                order_summary += f"The total pending buy order amount is ${buy_order_amount:.2f}."
        return order_summary

In [237]:
# check_pending_order.invoke({"user_id":"AB123"})

In [67]:
@tool
def check_earnings(user_id:str):
    """
    Check earning of user's trading account.
    :param user_id: the user's id
    :return: a string that describes what and how many stocks (equities, shares) the user is holding and what is their market value. What is the current profit or loss of the user.
    """
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT trading_account FROM user WHERE user_id = ?", (user_id,))
    trading_account = cursor.fetchone()[0]
    if trading_account is None:
        return "The client has no trading account with the bank. There is no trading information available."

    query = f"SELECT * FROM {trading_account}"
    cursor.execute(query)
    columns = [desc[0] for desc in cursor.description]
    
    df_all = pd.DataFrame(cursor.fetchall(), columns=columns)

    # Calculate the performance
    # Sort trades by date (safety)
    df_all = df_all.sort_values("date").reset_index(drop=True)

    # Dictionary to track each stock's holdings and P&L
    holdings = {}

    for _, row in df_all.iterrows():
        stock = row["stock"]
        volume = row["volume"]   # + for buy, - for sell
        total_amount = row["total_amount"]  # includes trading fee
        unit_cost = row["total_amount"]/row["volume"]
    
        if stock not in holdings:
            holdings[stock] = {"shares": 0, "cost_basis": 0.0, "realized_earning": 0.0}
    
        h = holdings[stock]
    
        if volume > 0:  # Buy
            h["shares"] += volume
            h["cost_basis"] += total_amount
    
        else:  # Sell
            shares_to_sell = -volume
            avg_cost = h["cost_basis"] / h["shares"] if h["shares"] > 0 else 0
            # Realized P&L from this sale
            realized = (unit_cost - avg_cost) * shares_to_sell
            h["realized_earning"] += realized
            # Reduce shares and cost basis
            h["shares"] += volume  # volume is negative
            h["cost_basis"] -= avg_cost * shares_to_sell
    
            # Safety check: reset if no shares remain
            if h["shares"] == 0:
                h["cost_basis"] = 0.0

    # Build result dict (only stocks with remaining shares)
    results = []
    search = TavilySearch(max_results=1, api_key=TAVILY_API_KEY)
    for stock, h in holdings.items():
        if h["shares"] > 0:
            avg_price = h["cost_basis"] / h["shares"]
            # get the current stock price
            current_price = None
            query = f"Check the current share price of {stock} in the US stock market."
            response = search.run(query)
            if response["results"][0]["content"]:
                final_response = llm.invoke(
                f'Here is the latest information of {stock} stock price: {response["results"][0]["content"]}, please extract the number of the stock price and only return the number in float format.')
                current_price = float(final_response.content)
            results.append({
                "stock": stock,
                "shares_remaining": h["shares"],
                "holding_price": round(avg_price, 2),
                "realized_earning": round(h["realized_earning"], 2),
                "current_price" : current_price,
                "unrealized_earning": round((current_price - avg_price)*h["shares"], 2) if current_price is not None else None
            })
    conn.close()

    # Create a natural language summary
    parts = []
    total_value = 0
    total_realized = 0
    
    for item in results:
        stock = item['stock']
        shares = item['shares_remaining']
        holding_price = item['holding_price']
        current_price = item['current_price']
        realized = item['realized_earning']
        holding_value = shares * current_price
        
        total_value += holding_value
        total_realized += realized
        
        parts.append(
            f"stock {stock} with {shares} shares at ${holding_price:.2f} per share, "
            f"the current price of the stock is ${current_price:.2f}, "
            f"the current holding value is ${holding_value:,.2f}, "
            f"and the realized earning is ${realized:,.2f}."
        )
    
    summary = "The user is holding:\n" + "\n".join(parts) + (
        f"\n\nIn total, the user's total holding value of all stocks is "
        f"${total_value:,.2f}, and the total realized earning is ${total_realized:,.2f}."
    )
    return summary, results[0]

In [68]:
check_earnings.invoke({"user_id":"AB123"})

("The user is holding:\nstock Adobe with 80 shares at $407.16 per share, the current price of the stock is $352.73, the current holding value is $28,218.40, and the realized earning is $72.30.\nstock Apple with 50 shares at $202.71 per share, the current price of the stock is $238.15, the current holding value is $11,907.50, and the realized earning is $505.11.\nstock Nvidia with 350 shares at $166.36 per share, the current price of the stock is $174.88, the current holding value is $61,208.00, and the realized earning is $469.78.\n\nIn total, the user's total holding value of all stocks is $101,333.90, and the total realized earning is $1,047.19.",
 {'stock': 'Adobe',
  'shares_remaining': 80,
  'holding_price': 407.16,
  'realized_earning': 72.3,
  'current_price': 352.73,
  'unrealized_earning': -4354.45})

In [57]:
stock = "Nike"
query = f"Check the current share price of {stock} in the US stock market."
response = search.run(query)
if response["results"][0]["content"]:
    final_response = llm.invoke(
    f'Here is the latest information of {stock} stock price: {response["results"][0]["content"]}, please extract the number of the stock price and only return the number in float format.')

print(final_response.content)
print(response["results"][0]["content"])

73.00
Join Us. Investors. NYSE NKE $73.00 -1.33. Investors. News, Events and ... Opening Price. $73.18. Closing Price. $73.03. Investment Calculator. Share. Email


In [65]:
portfolio = [
    {'stock': 'Adobe',
     'shares_remaining': 80,
     'holding_price': 407.16,
     'realized_earning': 72.3,
     'current_price': 352.73,
     'unrealized_earning': -4354.45},
    {'stock': 'Apple',
     'shares_remaining': 50,
     'holding_price': 202.71,
     'realized_earning': 505.11,
     'current_price': 238.15,
     'unrealized_earning': 1772.08},
    {'stock': 'Nvidia',
     'shares_remaining': 350,
     'holding_price': 166.36,
     'realized_earning': 469.78,
     'current_price': 174.88,
     'unrealized_earning': 2982.32}
]

# Build summary string
parts = []
total_value = 0
total_realized = 0

for item in portfolio:
    stock = item['stock']
    shares = item['shares_remaining']
    holding_price = item['holding_price']
    current_price = item['current_price']
    realized = item['realized_earning']
    holding_value = shares * current_price
    
    total_value += holding_value
    total_realized += realized
    
    parts.append(
        f"stock {stock} with {shares} shares at ${holding_price:.2f} per share, "
        f"the current price of the stock is ${current_price:.2f}, "
        f"the current holding value is ${holding_value:,.2f}, "
        f"and the realized earning is ${realized:,.2f}."
    )

summary = "The user is holding:\n" + "\n".join(parts) + (
    f"\n\nIn total, the user's total holding value of all stocks is "
    f"${total_value:,.2f}, and the total realized earning is ${total_realized:,.2f}."
)

print(summary)

The user is holding:
stock Adobe with 80 shares at $407.16 per share, the current price of the stock is $352.73, the current holding value is $28,218.40, and the realized earning is $72.30.
stock Apple with 50 shares at $202.71 per share, the current price of the stock is $238.15, the current holding value is $11,907.50, and the realized earning is $505.11.
stock Nvidia with 350 shares at $166.36 per share, the current price of the stock is $174.88, the current holding value is $61,208.00, and the realized earning is $469.78.

In total, the user's total holding value of all stocks is $101,333.90, and the total realized earning is $1,047.19.


In [357]:
def trade_stock(user_id: str, stock: str, action: Literal["buy", "sell"], volume: int, price: float):
    """
    Perform a trade (buy or sell stocks/securities/shares) on the user's trading account.
    Only day limit orders are supported.
    To submit the buy order request successfully, the user must have sufficient fund in the trading account.
    The sufficient fund means cash excluding the amount of other pending buy orders.
    Args:
        user_id: the user's id
        stock: the name of the stock/security/share to trade
        action: the type of trade, buy or sell
        volume: the volume of the trade
        price: the bid/asking price of the trade

    Returns:
        The status of the trade, and an update to the database if the trade is successfully submitted.
    """
    current_price = get_current_price(stock)
    if current_price.stock_name is None:
        return f"The stock you want to {action} is not available in the US stock market. We only support tradeing in the US stock market."
    else:
        stock = current_price.stock_name
    # Get user trading account information
    conn = connect(banking_data_db)
    cursor = conn.cursor()
    cursor.execute("SELECT trading_account FROM user WHERE user_id = ?", (user_id,))
    row = cursor.fetchone()
    if row is None:
        cursor.close()
        conn.close()
        return "No user found with this ID."
    trading_account = row[0]
    if trading_account is None:
        cursor.close()
        conn.close()
        return "The user has no trading account with the bank."

    # Trading information for both buy and sell orders
    # Trading history and trading account balance
    df_all = pd.read_sql_query(f"SELECT * FROM {trading_account}", conn).sort_values("date").reset_index(drop=True)
    # Pending orders
    query = "SELECT * FROM pending_orders WHERE trading_account = ?"
    df_orders = pd.read_sql_query(query, conn, params=(trading_account,)).reset_index(drop=True)
    # Trading amount
    trading_amount = price * volume
    trading_fee = trading_amount * 0.005
    # Current date and time
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Trade logic
    # Buy orders
    if action == "buy":
        # Get available funds for validation of buy orders
        # Get latest cash_end
        current_cash = 0
        if len(df_all)>0:
            current_cash = df_all["cash_end"].iloc[-1]
        # Get the pending buy order amount
        df_buy_orders = df_orders[df_orders["action"]=="buy"]
        dict_buy_orders = df_buy_orders.to_dict(orient="records")
        pending_buy_order_amount = 0
        if len(dict_buy_orders) > 0:
            pending_buy_order_amount = sum(item["total_amount"] for item in dict_buy_orders)
        
        # Get total amount and available fund for the buy order
        total_amount = trading_amount + trading_fee
        available_funds = current_cash - pending_buy_order_amount
        
        if price < current_price.stock_price*0.8:
            cursor.close()
            conn.close()
            return f"Your bid price falls below the permitted threshold of ${current_price.stock_price * 0.8:,.2f}. Kindly revise your buy order to comply with the requirements."
        elif total_amount > current_cash:
            cursor.close()
            conn.close()
            return (
                f"Insufficient funds: Your trading account does not currently hold sufficient funds to place this buy order.\n"
                f"Required amount (including applicable trading fees): ${total_amount:,.2f}.\n"             
                f"Available funds for trading: ${available_funds:,.2f}.\n"
                "Please ensure your account is adequately funded before submitting a new order."
            )
        else:
            cursor.execute(
                """
                INSERT INTO pending_orders (date, trading_account, stock, action, unit_price, volume, trading_amount, trading_fee, total_amount)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (now, trading_account, current_price.stock_name, "buy", price, volume, trading_amount, trading_fee, total_amount)
            )
            conn.commit()
            cursor.close()
            conn.close()
            return (
                f"Order Confirmation: Your request to purchase {volume} shares of {current_price.stock_name} at ${price:,.2f} per share has been successfully submitted.\n"
                f"Total amount reserved for settlement: ${total_amount:,.2f}, which includes a trading fee of ${trading_fee:,.2f}."
            )

    # Sell orders
    elif action == "sell":
        # Get available shares for validation of sell orders
        # Track holdings
        df_stock = df_all[df_all["stock"] == stock]
        current_holdings = 0
        if len(df_stock) > 0:
            current_holdings = df_stock["volume"].sum()
        # Get pending volume for sell orders
        df_sell_orders = df_orders[df_orders["action"]=="sell"]
        df_stock_sell_orders = df_sell_orders[df_sell_orders["stock"] == stock]
        pending_sell_order_volume = 0
        if len(df_stock_sell_orders) > 0:
            pending_sell_order_volume = abs(df_stock_sell_orders["volume"].sum())
        
        # Get total amount and available volume for the sell order
        total_amount = trading_amount - trading_fee
        available_volume = current_holdings - pending_sell_order_volume
        
        if available_volume < volume:
            cursor.close()
            conn.close()
            return (
                f"You do not currently hold a sufficient quantity of {current_price.stock_name} shares to place this sell order.\n"
                f"Available volume for trading: {int(available_volume)}.\n"
            )
        elif price > current_price.stock_price*1.2:
            cursor.close()
            conn.close()
            return f"Your asking price exceeds the allowable limit of ${current_price.stock_price * 1.2:,.2f}. Please adjust your sell order to align with the requirements."
        else:
            cursor.execute(
                """
                INSERT INTO pending_orders (date, trading_account, stock, action, unit_price, volume, trading_amount, trading_fee, total_amount)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (now, trading_account, current_price.stock_name, "sell", price, int(volume*(-1)), trading_amount*(-1), trading_fee, total_amount*(-1))
            )
            conn.commit()
            cursor.close()
            conn.close()
            return (
                f"Order Confirmation: Your request to sell {volume} shares of {current_price.stock_name} at ${price:,.2f} per share has been successfully submitted.\n"
                f"Estimated net proceeds from this transaction: ${total_amount:,.2f}, after deducting a trading fee of ${trading_fee:,.2f}."
            ) 

    return "The specified trade action is invalid. Kindly select either 'buy' or 'sell' to proceed with your transaction."

In [359]:
print(trade_stock(user_id="AB123", stock="Nvida", action= "sell", volume=30, price=210))

Like other stocks, NVDA shares are traded on stock exchanges, e.g. Nasdaq, Nyse, Euronext, and the easiest way to buy them is through an online stock broker. To
Yes
stock_name='NVIDIA' stock_price=176.67
NVIDIA
                  date   stock action  unit_price  volume  trading_amount  \
3  2025-06-03 00:00:00  NVIDIA    buy      141.22     200         28244.0   
5  2025-06-12 00:00:00  NVIDIA   sell      145.00    -200        -29000.0   
6  2025-06-18 00:00:00  NVIDIA    buy      145.48     100         14548.0   
7  2025-07-21 00:00:00  NVIDIA    buy      171.38     200         34276.0   
9  2025-08-01 00:00:00  NVIDIA    buy      173.72      50          8686.0   

   trading_fee  total_amount  cash_start    cash_end  
3       141.22      28385.22  59013.0825  30627.8625  
5       145.00     -28855.00  38843.3785  67698.3785  
6        72.74      14620.74  67698.3785  53077.6385  
7       171.38      34447.38  53077.6385  18630.2585  
9        43.43       8729.43  29270.7885  20541.358

In [311]:
conn = connect(banking_data_db)
user_id = "AB123"
stock = "Nvidia"
cursor = conn.cursor()
cursor.execute("SELECT trading_account FROM user WHERE user_id = ?", (user_id,))
row = cursor.fetchone()
trading_account = row[0]

df_all = pd.read_sql_query(f"SELECT * FROM {trading_account}", conn).sort_values("date").reset_index(drop=True)
df_stock = df_all[df_all["stock"] == stock]
holdings = 0
if len(df_stock) > 0:
    holdings = df_stock["volume"].sum()
print(df_stock)
print(holdings)
print(type(holdings))

                  date   stock action  unit_price  volume  trading_amount  \
3  2025-06-03 00:00:00  Nvidia    buy      141.22     200         28244.0   
5  2025-06-12 00:00:00  Nvidia   sell      145.00    -200        -29000.0   
6  2025-06-18 00:00:00  Nvidia    buy      145.48     100         14548.0   
7  2025-07-21 00:00:00  Nvidia    buy      171.38     200         34276.0   
9  2025-08-01 00:00:00  Nvidia    buy      173.72      50          8686.0   

   trading_fee  total_amount  cash_start    cash_end  
3       141.22      28385.22  59013.0825  30627.8625  
5       145.00     -28855.00  38843.3785  67698.3785  
6        72.74      14620.74  67698.3785  53077.6385  
7       171.38      34447.38  53077.6385  18630.2585  
9        43.43       8729.43  29270.7885  20541.3585  
350
<class 'numpy.int64'>
